In [1]:
# Hugging face testing
from transformers import AutoTokenizer, TFAutoModelForMaskedLM
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

model = TFAutoModelForMaskedLM.from_pretrained("bert-base-chinese")

/userhome/cs2/u3507348/anaconda3/envs/tensorflow_2_11/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-12 19:52:42.001991: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-12 19:53:19.425883: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-12 19:53:19.505531: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read fro

In [2]:
def testModel(sentence, k = 5):
    # Tokenize the input sentence
    inputs = tokenizer(sentence, return_tensors='tf')

    # Get the position of the [MASK] token
    mask_token_index = tf.where(inputs['input_ids'] == tokenizer.mask_token_id)[0, 1]

    # Generate model predictions for the masked token
    outputs = model(inputs['input_ids'], attention_mask=inputs['attention_mask'])
    predictions = outputs.logits[0, mask_token_index]

    # Convert the predictions to probabilities
    probabilities = tf.nn.softmax(predictions, axis=-1)

    # Get the top-k predicted tokens and their probabilities
    top_k_values, top_k_indices = tf.math.top_k(probabilities, k=k)
    predicted_tokens = tokenizer.convert_ids_to_tokens(top_k_indices.numpy())

    # Print the top-k predicted tokens and their probabilities
    print(f"Top-{k} predictions:")
    for i in range(k):
        print(f"{predicted_tokens[i]}: {top_k_values[i].numpy()}")
        
# Define the input sentence with [MASK] token
sentence = "今天的天氣很清[MASK]舒服"
testModel(sentence)

Top-5 predictions:
爽: 0.6484012007713318
新: 0.16201545298099518
涼: 0.08563422411680222
朗: 0.0348283126950264
淨: 0.006423946004360914


In [3]:
# Try fine tune it with own data
from datasets import load_dataset

dataset = load_dataset("csv", data_files="../dataset/set_01_02_new.csv", split='train')
dataset = dataset.train_test_split(test_size=0.2)

print(dataset)

dataset = dataset.filter(lambda example: all(example[key] is not None for key in example))

train_data = dataset['train']
test_data = dataset['test']

print(train_data[0])
print(test_data[0])

Found cached dataset csv (/userhome/cs2/u3507348/.cache/huggingface/datasets/csv/default-84069fde21d8afc3/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 4155
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 1039
    })
})


{'label': 0, 'text': 'much gave morning'}
{'label': 0, 'text': 'thank much skyped wit kz sura didnt get pleasure company hope good given ultimatum oh countin aburo enjoy message sent day ago'}


In [8]:
# Set up tokenizer
tokenized_train_data = tokenizer(train_data["text"], return_tensors="np", padding=True)

# Tokenizer returns a BatchEncoding, but we convert that to a dict for Keras
# tokenized_train_data = dict(tokenized_train_data)

# print(tokenized_train_data)

tokenized_train_data = tokenizer(train_data["text"], return_tensors="np", padding=True, truncation=True, max_length=512)
tokenized_test_data = tokenizer(test_data["text"], return_tensors="np", padding=True, truncation=True, max_length=512)

print(tokenized_train_data)
print(tokenized_test_data)

{'input_ids': array([[  101, 11677,  8370, ...,     0,     0,     0],
       [  101,  8385,  9064, ...,     0,     0,     0],
       [  101, 13233, 11333, ...,     0,     0,     0],
       ...,
       [  101,   163,  8583, ...,     0,     0,     0],
       [  101, 10614,  8196, ...,     0,     0,     0],
       [  101, 11694, 11460, ...,     0,     0,     0]]), 'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])}
{'input_ids': array([[  101, 13158,  8197, ...,     0,     0,     0],
       [  101,  8993, 10123, ...,     0,     0,     0],
       [  101, 10353, 10919, ...,     0,     0,     0],
      

In [22]:
from tensorflow.keras.optimizers import Adam
import numpy as np

def tokenize_dataset(data):
    # Keys of the returned dictionary will be added to the dataset as columns
    return tokenizer(data["text"])

dataset = train_data.map(tokenize_dataset)

tf_dataset = model.prepare_tf_dataset(dataset, batch_size=92, shuffle=True, tokenizer=tokenizer)

model.compile(optimizer=Adam(3e-5))

model.fit(tf_dataset)


Loading cached processed dataset at /userhome/cs2/u3507348/.cache/huggingface/datasets/csv/default-84069fde21d8afc3/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-853fa0dc895715a0.arrow
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.
2023-04-12 20:33:57.285011: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 579.66MiB (rounded to 607822848)requested by op tf_bert_for_masked_lm/bert/encoder/layer_._1/attention/self/MatMul
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-04-12 20:33

ResourceExhaustedError: Graph execution error:

Detected at node 'tf_bert_for_masked_lm/bert/encoder/layer_._1/attention/self/MatMul' defined at (most recent call last):
    File "/userhome/cs2/u3507348/anaconda3/envs/tensorflow_2_11/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/userhome/cs2/u3507348/anaconda3/envs/tensorflow_2_11/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/userhome/cs2/u3507348/anaconda3/envs/tensorflow_2_11/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/userhome/cs2/u3507348/anaconda3/envs/tensorflow_2_11/lib/python3.9/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/userhome/cs2/u3507348/anaconda3/envs/tensorflow_2_11/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/us

In [12]:
from transformers import TFTrainer, TFTrainingArguments
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

# Convert NumPy arrays to TensorFlow tensors
# train_inputs = {k: tf.convert_to_tensor(v) for k, v in tokenized_train_data.items()}
# train_labels = tf.convert_to_tensor(train_data["label"])

# test_inputs = {k: tf.convert_to_tensor(v) for k, v in tokenized_test_data.items()}
# test_labels = tf.convert_to_tensor(test_data["label"])

# train_dataset = tf.data.Dataset.from_tensor_slices((train_inputs, train_labels)).shuffle(1000).batch(16)
# test_dataset = tf.data.Dataset.from_tensor_slices((test_inputs, test_labels)).batch(16)

# Prepare dataset for training
train_dataset = tf.data.Dataset.from_tensor_slices((dict(tokenized_train_data), train_data["label"])).shuffle(1000).batch(16)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(tokenized_test_data), test_data["label"])).batch(16)

# Training arguments
training_args = TFTrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    logging_dir='./logs',
    logging_steps=50,
    load_best_model_at_end=True,
    evaluation_strategy="steps"
)

# Define metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = tf.math.argmax(logits, axis=-1).numpy()
    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions)
    recall = recall_score(labels, predictions)
    f1 = f1_score(labels, predictions)
    return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}

# Trainer
trainer = TFTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)


In [13]:
# Train
history = trainer.train()

# Plot performance
plt.plot(history.epoch, history.history['accuracy'], label='accuracy')
plt.plot(history.epoch, history.history['precision'], label='precision')
plt.plot(history.epoch, history.history['recall'], label='recall')
plt.plot(history.epoch, history.history['f1'], label='f1')
plt.xlabel('Epochs')
plt.ylabel('Metrics')
plt.legend()
plt.show()

2023-04-12 20:08:24.057826: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_4"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
input: "Placeholder/_2"
input: "Placeholder/_3"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_INT64
      type: DT_INT64
      type: DT_INT64
      type: DT_INT32
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 4153
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\025TensorSliceDataset:37"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 512
        }
      }
      shape {
        dim {
          size: 512
        }
      }
      shape {
        dim {
          size: 512
        }
      }
      sh

ValueError: in user code:

    File "/userhome/cs2/u3507348/anaconda3/envs/tensorflow_2_11/lib/python3.9/site-packages/transformers/trainer_tf.py", line 710, in distributed_training_steps  *
        self.args.strategy.run(self.apply_gradients, inputs)
    File "/userhome/cs2/u3507348/anaconda3/envs/tensorflow_2_11/lib/python3.9/site-packages/transformers/trainer_tf.py", line 653, in apply_gradients  *
        gradients = self.training_step(features, labels, nb_instances_in_global_batch)
    File "/userhome/cs2/u3507348/anaconda3/envs/tensorflow_2_11/lib/python3.9/site-packages/transformers/trainer_tf.py", line 636, in training_step  *
        per_example_loss, _ = self.run_model(features, labels, True)
    File "/userhome/cs2/u3507348/anaconda3/envs/tensorflow_2_11/lib/python3.9/site-packages/transformers/trainer_tf.py", line 755, in run_model  *
        outputs = self.model(features, labels=labels, training=training)[:2]
    File "/userhome/cs2/u3507348/anaconda3/envs/tensorflow_2_11/